In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import os
path='C:\\Users\\aemra\\Documents\\GitHub\\Wrangle-OpenStreetMap-Data'
os.chdir(path)
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

created = {'version': None, 'timestamp': None, 'changeset': None, 'user': None,
           'uid': None}
L = []
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]



def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        created = {}
        for e in element.attrib.keys():
            
            if e in CREATED:
                
                created[e] = element.attrib[e]
                
                
            elif  element.attrib[e] == element.get('lat') or element.attrib[e] == element.get('lon'):
                pos = []
                pos.append(float(element.get('lat')))
                pos.append(float(element.get('lon')))
                node['pos'] = pos

            else:
                node[e] = element.get(e)
                
                
                node['type'] = element.tag

        node['created'] = created
            

        node_refs = []
        address = {}
        for subtag in element:
            if subtag.tag == 'tag':
                if re.search(problemchars, subtag.get('k')):
                    pass
                elif re.search(r'\w+:\w+:\w+', subtag.get('k')):
                    pass
                elif subtag.get('k').startswith('addr:'):
                    address[subtag.get('k')[5:]] = subtag.get('v')
                    node['address'] = address
                else:
                    node[subtag.get('k')] = subtag.get('v')

            else:
                if subtag.tag == 'nd':
                    
                    node_refs.append(subtag.get('ref'))
                    
                else:
                    pass

        
        if node_refs:
            node['node_refs'] = node_refs
            

        
        return node
    else:
        return None



def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map('houston_texas.osm', True)
    pprint.pprint(data)
    
    
    
if __name__ == "__main__":
    test()

MemoryError: 